In [82]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
np.random.seed(100)
tf.random.set_seed(100)

df = pd.read_csv('combined_dataset.csv')

df.set_index(df.columns[0], inplace=True)

duplicate_columns = df.columns[df.columns.duplicated()]
df = df.drop(columns=duplicate_columns)

# Using 'NASDAQ_Volatility' as the target variable
X = df.drop(columns=['NASDAQ_Volatility']).values
y = df['NASDAQ_Volatility'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


n_timesteps = 1
X_train = X_train.reshape((X_train.shape[0], n_timesteps, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], n_timesteps, X_test.shape[1]))

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_timesteps, X_train.shape[2])))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')


model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)


y_pred = model.predict(X_test)


r_squared = r2_score(y_test, y_pred)
print(f'R-squared on Test Set: {r_squared}')


Epoch 1/50
510/510 [==============================] - 1s 661us/step - loss: 0.0084
Epoch 2/50
510/510 [==============================] - 0s 634us/step - loss: 0.0018
Epoch 3/50
510/510 [==============================] - 0s 621us/step - loss: 8.6212e-04
Epoch 4/50
510/510 [==============================] - 0s 612us/step - loss: 3.4111e-04
Epoch 5/50
510/510 [==============================] - 0s 649us/step - loss: 2.0664e-04
Epoch 6/50
510/510 [==============================] - 0s 635us/step - loss: 1.5183e-04
Epoch 7/50
510/510 [==============================] - 0s 624us/step - loss: 1.2606e-04
Epoch 8/50
510/510 [==============================] - 0s 621us/step - loss: 1.0146e-04
Epoch 9/50
510/510 [==============================] - 0s 602us/step - loss: 1.0608e-04
Epoch 10/50
510/510 [==============================] - 0s 605us/step - loss: 1.1403e-04
Epoch 11/50
510/510 [==============================] - 0s 599us/step - loss: 6.4391e-05
Epoch 12/50
510/510 [============================

In [85]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import Dropout, Bidirectional

np.random.seed(42)
tf.random.set_seed(42)


df = pd.read_csv('combined_dataset.csv')


df.set_index(df.columns[0], inplace=True)

duplicate_columns = df.columns[df.columns.duplicated()]
df = df.drop(columns=duplicate_columns)

X = df.drop(columns=['NASDAQ_Volatility']).values
y = df['NASDAQ_Volatility'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


n_timesteps = 1 
X_train = X_train.reshape((X_train.shape[0], n_timesteps, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], n_timesteps, X_test.shape[1]))


model = Sequential()

model.add(Bidirectional(LSTM(50, activation='relu', return_sequences=True), input_shape=(n_timesteps, X_train.shape[2])))

model.add(LSTM(38, activation='relu', return_sequences=False))
#model.add(Dropout(0.2))

model.add(Dense(30, activation='relu'))

model.add(Dense(20, activation='relu'))
#model.add(Dropout(0.2))



model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

y_pred = model.predict(X_test)



# Calculate R-squared
r_squared = r2_score(y_test, y_pred)
print(f'R-squared on Test Set: {r_squared}')


Epoch 1/50
510/510 [==============================] - 2s 2ms/step - loss: 2.0246e-05
Epoch 2/50
510/510 [==============================] - 1s 2ms/step - loss: 6.5939e-06
Epoch 3/50
510/510 [==============================] - 1s 2ms/step - loss: 1.4318e-05
Epoch 4/50
510/510 [==============================] - 1s 2ms/step - loss: 2.1753e-06
Epoch 5/50
510/510 [==============================] - 1s 2ms/step - loss: 1.2150e-05
Epoch 6/50
510/510 [==============================] - 1s 2ms/step - loss: 2.1613e-06
Epoch 7/50
510/510 [==============================] - 1s 2ms/step - loss: 1.0081e-06
Epoch 8/50
510/510 [==============================] - 1s 2ms/step - loss: 4.1899e-07
Epoch 9/50
510/510 [==============================] - 1s 2ms/step - loss: 2.3418e-07
Epoch 10/50
510/510 [==============================] - 1s 2ms/step - loss: 2.1104e-07
Epoch 11/50
510/510 [==============================] - 1s 2ms/step - loss: 2.1058e-07
Epoch 12/50
510/510 [==============================] - 1s 2ms/s